# Important Imports

In [10]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Business Question: Cluster pationts in a similer clusters
### Every row reprisents a pationt

# Load Data

In [8]:
patients = pd.read_csv("../synthetichealth/Data/patients.csv")
conditions = pd.read_csv("../synthetichealth/Data/conditions.csv")
careplans = pd.read_csv("../synthetichealth/Data/careplans.csv")
observations = pd.read_csv("../synthetichealth/Data/observations.csv")
encounters = pd.read_csv("../synthetichealth/Data/encounters.csv")
procedures = pd.read_csv("../synthetichealth/Data/procedures.csv")
immunizations = pd.read_csv("../synthetichealth/Data/immunizations.csv")
medications = pd.read_csv("../synthetichealth/Data/medications.csv")

In [11]:
patients.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,d171d808-1f31-4ad3-aba5-e03a2fa921c7,1983-04-03,NaN,999-83-6585,S99914532,X18544570X,Mr.,Marco Antonio298,Hurtado459,NaN,NaN,M,hispanic,mexican,M,La Paz Baja California MX,552 Rippin Port,Revere,Massachusetts,Suffolk County,2151.0,42.493847,-70.928585,748893.99,2679.68
1,3603cd65-53a3-424b-bc82-76204326510d,1977-06-24,NaN,999-62-4132,S99987902,X33240404X,Mr.,Guillermo498,Trantow673,NaN,NaN,M,white,irish,M,Southbridge Massachusetts US,1027 Satterfield Spur Unit 67,Richmond,Massachusetts,Berkshire County,NaN,42.390136,-73.324373,870371.41,5316.00
2,bd512b17-9e68-4b0e-8c5b-980007ecdee1,1991-10-21,NaN,999-65-7753,S99933589,X60093276X,Mr.,Hernán834,Ojeda263,NaN,NaN,M,black,dominican,M,Marigot Saint Andrew Parish DM,257 Bradtke Mill Unit 35,Newton,Massachusetts,Middlesex County,2458.0,42.419939,-71.128108,607565.78,2855.62
3,21dd496c-2fec-4c9c-a0fe-c11ad5b2cdea,2006-12-17,NaN,999-72-6694,NaN,NaN,NaN,Francisco472,Prosacco716,NaN,NaN,NaN,white,portuguese,M,Funchal Madeira PT,729 Nienow Gateway,Townsend,Massachusetts,Middlesex County,1469.0,42.698664,-71.614920,323891.14,2919.84
4,ab3533aa-8230-4608-9f06-4f4673123620,1977-10-14,NaN,999-85-1774,S99964476,X82206403X,Mrs.,Elizabet136,Marks830,NaN,Morissette863,M,white,french_canadian,F,Haverhill Massachusetts US,243 Bartoletti Neck Suite 63,Sturbridge,Massachusetts,Worcester County,1507.0,42.110219,-72.049690,870544.88,7831.98


In [12]:
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2016-04-14,2016-04-28,d171d808-1f31-4ad3-aba5-e03a2fa921c7,0b4374e8-838d-4289-b491-f44d51360790,444814009,Viral sinusitis (disorder)
1,1995-08-18,NaN,3603cd65-53a3-424b-bc82-76204326510d,61f6867a-9e58-48e6-87d0-6d9495a81a79,59621000,Hypertension
2,2010-02-12,2010-02-19,3603cd65-53a3-424b-bc82-76204326510d,ae97dfe0-c619-44c1-a687-f215d7fac984,10509002,Acute bronchitis (disorder)
3,1998-10-19,2009-10-26,bd512b17-9e68-4b0e-8c5b-980007ecdee1,e0213a36-1b17-45c9-a233-31e1042e7b28,192127007,Child attention deficit disorder
4,2009-12-14,NaN,bd512b17-9e68-4b0e-8c5b-980007ecdee1,31bd0967-a973-4eaf-bf40-5ca6eb341eb7,15777000,Prediabetes


In [13]:
careplans.head()

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d6787192-da1d-48ae-a051-89216c95d864,1995-08-18,NaN,3603cd65-53a3-424b-bc82-76204326510d,61f6867a-9e58-48e6-87d0-6d9495a81a79,443402002,Lifestyle education regarding hypertension,59621000.0,Hypertension
1,336f5314-bed2-487a-bb08-3d8e364fbe7f,1998-10-19,2009-10-26,bd512b17-9e68-4b0e-8c5b-980007ecdee1,e0213a36-1b17-45c9-a233-31e1042e7b28,386522008,Overactivity/inattention behavior management,192127007.0,Child attention deficit disorder
2,112010d4-0982-4aeb-9753-2675fa40d291,2010-02-12,2011-09-09,3603cd65-53a3-424b-bc82-76204326510d,ae97dfe0-c619-44c1-a687-f215d7fac984,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
3,19a665ed-215b-4bc6-bec6-9cedb6e5c2e5,2009-12-14,NaN,bd512b17-9e68-4b0e-8c5b-980007ecdee1,31bd0967-a973-4eaf-bf40-5ca6eb341eb7,698360004,Diabetes self management plan,15777000.0,Prediabetes
4,3ef8ba2a-4866-45e6-ad76-6ee515525e78,2014-09-12,NaN,3603cd65-53a3-424b-bc82-76204326510d,88ed9c64-9ab0-46da-9a10-4f896bd57b92,698360004,Diabetes self management plan,15777000.0,Prediabetes
